<a href="https://colab.research.google.com/github/Shahriar-Tonmoy/Bangla_writtendigit_recognization/blob/master/SC_Assignment2_Exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown
!gdown --id '1-0Mc5nKDRGPEi5FBTA0hGLS6OgvTgDMq'

In [ ]:
!rm './SC_Dataset_Assignment2.zip'

In [ ]:
import matplotlib.pyplot as plt
import warnings
import time,sys
import copy

import pandas as pd
import numpy as np
import cv2
import os
import pathlib
import zipfile
import torch
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.optim as optim
import pathlib
import shutil
from pathlib import Path
from collections import Counter
!pip install torchsummary
from torchsummary import summary

!pip install torchviz
from torchviz import make_dot, make_dot_from_trace

warnings.filterwarnings('ignore')
torch.manual_seed(0)

In [ ]:
base_dir ='./SC_Dataset_Assignment2' 
BATCH_SIZE = 32

IMAGE_SIZE = 28
LEARNING_RATE = 0.01
TEST_SIZE = 0.2
OUTPUT_DIM=10

In [ ]:
# transform = transforms.Compose([

#                                 transforms.Grayscale(),  
#                                 transforms.Resize(IMAGE_SIZE),  
#                                 transforms.CenterCrop(IMAGE_SIZE), 
#                                 transforms.ToTensor(),
#                                ])

 
# dataset = torchvision.datasets.ImageFolder(base_dir, transform=transform)
# n = len(dataset) 
# n_test = int(TEST_SIZE * n) 
# trainDataset, validDataSet = torch.utils.data.random_split(dataset,[n - n_test,n_test])
# trainloader = torch.utils.data.DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,) 
# validationloader = torch.utils.data.DataLoader(validDataSet, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,) 
# print("Length of the trainloader:", len(trainloader ) * BATCH_SIZE)
# print("Length of the validationloader:", len(validationloader ) * BATCH_SIZE)


In [ ]:
trainDataset = torchvision.datasets.FashionMNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  # Normalize the image to [0-1] from [0-255]
                            download=True)

validDataSet = torchvision.datasets.FashionMNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
MAKING DATASET ITERABLE
'''

trainloader = torch.utils.data.DataLoader(dataset=trainDataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)   # It's better to shuffle the whole training dataset! 

validationloader = torch.utils.data.DataLoader(dataset=validDataSet, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False)  

print("Length of the trainloader:", len(trainloader ) * BATCH_SIZE)
print("Length of the validationloader:", len(validationloader ) * BATCH_SIZE)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class LIN_MODEL_3(torch.nn.Module): 
    def __init__(self,outDim):
        super(LIN_MODEL_3, self).__init__()

        self.fc_1 = torch.nn.Linear(784, 512)
        self.fc_2 = torch.nn.Linear(512, 256)
        self.fc_3 = torch.nn.Linear(256,128)
        self.fc_4 = torch.nn.Linear(128,64)
        self.fc_6 = torch.nn.Linear(64, outDim)
 
    def forward(self, x):

        x = x.view(-1, 28 * 28) 
        x = torch.nn.functional.relu(self.fc_1(x))
        x = torch.nn.functional.relu(self.fc_2(x))
        x = torch.nn.functional.relu(self.fc_3(x))
        x = torch.nn.functional.relu(self.fc_4(x))
        x = torch.nn.functional.relu(self.fc_6(x))

        return x
        
model_2 = LIN_MODEL_3(OUTPUT_DIM).to(device)

summary( model_2, input_size=(1, 28, 28))
print(model_2)

In [ ]:
def save_model(
               MODEL_USED,
               SAVEPATH,
               epoch, 
               batch_size, 
               model,
               optimizer,
               image_size,
               tranning_loss=[],
               tranning_acc=[],
               validation_loss=[],
               validation_acc=[],
               learning_rate=0.001,
               meta_data=None):
  SAVEPATH += f"{MODEL_USED}-checkpoint-epoch-{epoch}.pt"
  save_obj = {
       'MODEL_USED':MODEL_USED,
       'batch_size':batch_size,
       'epoch': epoch,
       'model_full': model,
       'optimizer_full': optimizer,
       'model_state': model.state_dict(),
       'optimizer_state': optimizer.state_dict(),
       'image_size': image_size,
       'tranning_loss': tranning_loss,
       'tranning_acc': tranning_acc,
       'validation_loss': validation_loss,
       'validation_acc': validation_acc,
       'learning_rate':learning_rate,
       'meta_data':meta_data
       }

  torch.save(save_obj, SAVEPATH)

In [ ]:
def train_model(start, end, 
                model_used ,
                model_save_path, 
                model, 
                criterion, 
                optimizer, 
                dataloaders,
                testloaders , 
                lernRate=0.01,
                all_tranning_loss=[], all_validation_loss=[], all_tranning_accuracy=[], all_validation_accuracy=[]):
    since = time.time()
    num_epochs = end
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    phase = 'train'
    steps = 0

    for epoch in range(start,num_epochs):
          model.train()
          phase = 'tranning'
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)
          running_loss = 0.0
          running_corrects = 0

          for i,(inputs, labels) in enumerate(dataloaders):
                
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)
                

                print_val = f"Epoch: {epoch}/{num_epochs-1} Steps {steps} \t"
                print_val += f"running_loss : {(loss.item()):.6f}\t"
                print_val += f"running_corrects : {torch.sum(preds == labels.data)}\t"  
                print_val += f"total_corrects : {running_corrects}\t"  
                sys.stdout.write('\r' + str(print_val))
                steps += 1
          
          
          steps = 0
          epoch_loss = running_loss / len(dataloaders)
          epoch_acc = running_corrects.double().item() /len(dataloaders.dataset)
          all_tranning_loss.append(loss.item())
          all_tranning_accuracy.append(epoch_acc)
          
          
          print("\n")
          print("----------------------------Tranning Summary----------------------")
          print('{} Tranning Avg. Loss: {:.4f} Tranning Avg. Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
          print("-"*70)
          print("\n")

          print("Start Validation")
          model.eval()
          with torch.no_grad():
              phase = "validation"
              running_loss = 0.0
              running_corrects = 0
              for i,(inputs, labels) in enumerate(testloaders):
                  inputs, labels = inputs.to(device), labels.to(device)
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)
                    
                  running_loss += loss.item()
                  running_corrects += torch.sum(preds == labels.data)  

                  print_val = f"Steps {i} \t"
                  print_val += f"validation_running_loss : {(loss.item()):.6f}\t"
                  print_val += f"validation_running_corrects : {torch.sum(preds == labels.data)}\t"  
                  print_val += f"validation_total_corrects : {running_corrects}\t"  
                  sys.stdout.write('\r' + str(print_val))

              epoch_val_loss = running_loss / len(testloaders)
              epoch_val_acc = running_corrects.double().item() /len(testloaders.dataset)
              all_validation_loss.append(epoch_val_loss)
              all_validation_accuracy.append(epoch_val_acc)

              print()
              print("----------------------------Validation Summary-----------------")
              print('{} Validation Avg. Loss: {:.4f} Validation Avg. Acc: {:.4f}'.format(
                    phase, epoch_val_loss, epoch_val_acc))
              print("------------------------------------------------------------")


          model.train()
          
          print("-------Start Model Save----\n\n")
          save_model(model_used,
                     model_save_path,
                     epoch, 
                     len(dataloaders),
                     model,
                     optimizer,
                     IMAGE_SIZE,
                     tranning_loss=all_tranning_loss,
                     tranning_acc= all_tranning_accuracy,
                     validation_loss=all_validation_loss,
                     validation_acc=all_validation_accuracy,
                     learning_rate=lernRate)
   
      
    print("Complete Train")


    print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return 

In [ ]:
!mkdir './Model_2/'

In [ ]:
START=0
END=300
model_save_path = './Model_2/' 
model_used= 'EXPERIMENT_2_MODEL_2'
model_ft = model_2


optimizer_ft = optimizer
trainloader= trainloader
testloader = validationloader

In [ ]:
train_model(START, END,model_used, model_save_path, model_ft, criterion, optimizer_ft, trainloader, testloader)

In [ ]:
!wget 'https://www.kaggleusercontent.com/kf/74001471/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..eK9Z51O5blp1x3n91U0GdA.6uHRztoipwSGYYKJV2B66rGf5rblTIsEazbvhKWGApNX5m6Imr_r5WsuYGwOtRhHeUwz-8IVw7aC8JVKnhSSPZoPM_J8895VyETZWaprmOvHxEQd4AW0JViCxq2Zh_yVShvPgYq1sn4HW6JQa1acUTAaR9DtL_YRa3DBSzSq6nJy_MvN7uFNl7d-q4lEtEDyT6ykDtwa0YsEU9WTFts2o14sk65Sqeav72GtyZqTQ9oUCHR4XWIXIVRTp_UCmLzcTuYm3kudHe3l1KO7QlcsMIBdI0_aBAifXE86jfdCLxNz_D8vecyBIcWLOdRAYfjAclWJvoVZy9T2heBZuENcKf1RBDqzFAHsbS6mHXNFQDcuCpRFFsVTHuw73OBl6ZZ68hidN4wmE9mNvlzaP2k110Zp0nqWdTDUpPnAfT0qnCb_EHajlYF5Xt8QuR_tDz7CEMIDvNW_LGGDwCYhcMwdfXjgO6hjr3apeFcf7ASrUHBJBE5zw0Deu1JQsyzhHulozoWJJe4LVhxEHFVHM_sMNE7NMNyNCY040JILcXQi5iW8b9AqObVOB-BIPajSEHJVxYngbWQNr6-TNlv8uZktdiU78xYvXXwuMk8ic_xdZoITfgNlSx2cQl4wwmOmEqnLxB9xwPCPtR7R2QFJIGaCcA4RXF1Fpd1LEGWUNpW21kk.z4hUXhwzDWEEHI11TEtooA/Model_2.zip'

In [ ]:
!cp '/content/Model_2.zip' '/content/drive/MyDrive/SC_Assignment2'

In [ ]:
!unzip '/content/drive/MyDrive/SC_Assignment2/Model_2.zip'

In [ ]:
import glob,sys,os
import matplotlib.pyplot as plt
import warnings
import numpy as np
import seaborn as sn
import pandas as pd
import pathlib
import zipfile
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch.optim as optim
import time,sys
import copy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')

In [ ]:
MODEL_LOAD_PATH = '/content/Model_2/EXPERIMENT_2_MODEL_2-checkpoint-epoch-299.pt'
model = torch.load(MODEL_LOAD_PATH,map_location='cpu')
model

In [ ]:
_tranning_loss = model['tranning_loss']
_tranning_acc = model['tranning_acc']
_validation_loss = model['validation_loss']
_validation_acc = model['validation_acc']

In [ ]:
plt.figure(figsize=(10,5))
plt.title("LOSS Experiment_2_Mydataset")
plt.plot(_tranning_loss,label="Tranning Loss")
plt.plot(_validation_loss,label="Validation Loss")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Accuracy Experiment_2_Mydataset")
plt.plot(_tranning_acc,label="Tranning Accuracy")
plt.plot(_validation_acc,label="Validation Accuracy")
plt.xlabel("iterations")
plt.ylabel("Accuracy")
plt.legend()
plt.show()